In [22]:
# import nisnap
import pyxnat
import xnat
from pyxnat import Interface
import pandas as pd
import nibabel as nib
from tqdm import tqdm


In [14]:
my_pyxnat = Interface(server="https://xnat.crmbm.univ-amu.fr",
                    user='hdary')

In [15]:
project = 'TERRA_BRAIN_EPINOV'

In [28]:
my_pyxnat.select('xnat:projectData').array.experiments(columns=['ID', 'project']).data


AttributeError: 'Search' object has no attribute 'array'

In [43]:
def get_resources(my_pyxnat, project):

    resources = []
    experiments = my_pyxnat.array.experiments(project_id=project, columns=['ID', 'project', 'label', 'subject_ID']).data
    # For each experiments fetch all the resources associated with it
    for exp in tqdm(experiments):
        j = my_pyxnat.get('{}/{}'.format(exp['URI'], 'resources')).json()
        exp_res = j['ResultSet']['Result']

        if exp_res:
            resources.extend([[exp['project'], exp['ID'], exp['label'], exp['subject_ID'],
                               r['xnat_abstractresource_id'],
                               r['label']] for r in exp_res])
        else:
            resources.append([exp['project'], exp['ID'],  exp['label'], exp['subject_ID'],
                              'No Data', 'No Data'])
    return resources

In [ ]:
def get_resources(my_pyxnat, project):
for sub in my_pyxnat.select.project(project).subjects():



In [45]:
resources = get_resources(my_pyxnat, project)

  0%|          | 0/34 [00:00<?, ?it/s]

100%|██████████| 34/34 [00:00<00:00, 65.63it/s]


In [46]:
resources

[['TERRA_BRAIN_EPINOV',
  'XNAT_CRMBM_E00111',
  '202201140838',
  'XNAT_CRMBM_S00106',
  'No Data',
  'No Data'],
 ['TERRA_BRAIN_EPINOV',
  'XNAT_CRMBM_E00112',
  '202304280912',
  'XNAT_CRMBM_S00113',
  'No Data',
  'No Data'],
 ['TERRA_BRAIN_EPINOV',
  'XNAT_CRMBM_E00113',
  '202304071208',
  'XNAT_CRMBM_S00123',
  'No Data',
  'No Data'],
 ['TERRA_BRAIN_EPINOV',
  'XNAT_CRMBM_E00114',
  '202306291315',
  'XNAT_CRMBM_S00111',
  'No Data',
  'No Data'],
 ['TERRA_BRAIN_EPINOV',
  'XNAT_CRMBM_E00115',
  '202107020849',
  'XNAT_CRMBM_S00107',
  'No Data',
  'No Data'],
 ['TERRA_BRAIN_EPINOV',
  'XNAT_CRMBM_E00116',
  '202308220902',
  'XNAT_CRMBM_S00110',
  'No Data',
  'No Data'],
 ['TERRA_BRAIN_EPINOV',
  'XNAT_CRMBM_E00117',
  '202204131119',
  'XNAT_CRMBM_S00109',
  '24167',
  'FREESURFER'],
 ['TERRA_BRAIN_EPINOV',
  'XNAT_CRMBM_E00117',
  '202204131119',
  'XNAT_CRMBM_S00109',
  '24169',
  'Atlas'],
 ['TERRA_BRAIN_EPINOV',
  'XNAT_CRMBM_E00117',
  '202204131119',
  'XNAT_CRMBM_S001

In [ ]:
dico_resources = {}
freesurfer_res_list = []
atlas_res_list = []

for sub in my_pyxnat.select.project(project).subjects():
    for exp in sub.experiments():
        for res in exp.resources():
            if not os.path.exists()
            if res.label() == 'FREESURFER':
                freesurfer_res_list.append(res)
            if res.label() == 'Atlas':
                neudorfer_res_list.append(res)

In [ ]:
freesurfer_list

[<Resource Object> 24167 `FREESURFER` (346 files 396.10 MB),
 <Resource Object> 24165 `FREESURFER` (320 files 352.46 MB),
 <Resource Object> 24164 `FREESURFER` (264 files 349.32 MB)]

In [ ]:
freesurfer_list[0].aseg()

,measurement,region,value
0,Volume_mm3,BrainSegVol,1022580.374364
1,Volume_mm3,BrainSegVolNotVent,980107.281317
2,Volume_mm3,VentricleChoroidVol,36647.69615
3,Volume_mm3,lhCortexVol,204580.125985
4,Volume_mm3,rhCortexVol,213219.579237
...,...,...,...
419,normRange,CC_Posterior,98.0000
420,normRange,CC_Mid_Posterior,69.0000
421,normRange,CC_Central,96.0000
422,normRange,CC_Mid_Anterior,110.0000


In [ ]:
freesurfer_list[0].aparc()

,side,measurement,region,value
0,None,Volume_mm3,BrainSegVol,1022580.374364
1,None,Volume_mm3,BrainSegVolNotVent,980107.281317
2,None,Volume_mm3,BrainSegVolNotVentSurf,980107.281317
3,None,Volume_mm3,CortexVol,417799.705223
4,None,Volume_mm3,SupraTentorialVol,907305.992304
...,...,...,...,...
627,right,CurvInd,supramarginal,6.3
628,right,CurvInd,frontalpole,0.8
629,right,CurvInd,temporalpole,1.0
630,right,CurvInd,transversetemporal,0.4


In [ ]:
def extract_qmri(lut, subject, my_roi, t1, stats=None, r2s='', qsm='', na='', diff=''):

    # Load ROI labels and names
    lut = pd.read_csv(lut)


    # Load aseg stats output for eTIV
    try:
        aseg = pd.read_csv(stats,delimiter="\t")
        aseg = aseg.set_index('Measure:volume')
        etiv = aseg.loc['sub-{}'.format(subject)]['EstimatedTotalIntraCranialVol']
    except:
        etiv = -999

    # Load thalamic parcellation
    seg_file = nib.load(my_roi) 
    seg_img = seg_file.get_fdata()

    # Load T1 data
    t1_file = nib.load(t1)  
    t1_img = t1_file.get_fdata() 

    if ((r2s != '')) & (qsm != ''):
        megre = True

        # Load R2s data
        r2s_file = nib.load(r2s)
        r2s_img = r2s_file.get_fdata()

        # Load QSM data
        qsm_file = nib.load(qsm)  
        qsm_img = qsm_file.get_fdata()    
    else:
        megre = False   

    # Get voxel volume
    zooms = seg_file.header.get_zooms()
    voxel_mm3 = np.prod(zooms)    

    # Compute volume, mean R2s and T1 for each ROI
    d = []
    for l in range(0,len(lut)):
        volume_mm   = np.sum(np.isin(seg_img, lut.iloc[l]['label'], assume_unique=True)) * voxel_mm3
        volume_perc = (volume_mm / etiv) * 100
        t1     = np.nanmean(t1_img[np.isin(seg_img, lut.iloc[l]['label'], assume_unique=True)])

        if megre == True:
            r2s    = np.nanmean(r2s_img[np.isin(seg_img, lut.iloc[l]['label'], assume_unique=True)])
            qsm    = np.nanmean(qsm_img[np.isin(seg_img, lut.iloc[l]['label'], assume_unique=True)])
        else:
            r2s = -999
            qsm = -999

        data = ['sub-'+ subject,
                etiv,    
                lut.iloc[l]['label'],
                lut.iloc[l]['roi'],
                volume_mm,
                volume_perc,
                t1,
                r2s,
                qsm]

        columns=['Subject', 'eTIV', 'Label', 'ROI', 'Volume (mm)', 'Volume (%)', 'T1', 'R2s', 'QSM']
        # columns=['Subject', 'Label', 'ROI', 'Hemisphere', 'Volume (mm)', 'T1', 'R2s', 'QSM']

        if 'side' in subject_info.iloc[0]:
            data.append(subject_info.iloc[0]['side'])
            columns.append('Side')
        if 'type' in subject_info.iloc[0]:
            data.append(subject_info.iloc[0]['type'])
            columns.append('Type')
        if 'sex' in subject_info.iloc[0]:
            data.append(subject_info.iloc[0]['sex'])
            columns.append('Sex')
        if 'age' in subject_info.iloc[0]:
            data.append(subject_info.iloc[0]['age'])
            columns.append('Age')

        d.append(data)

    # Convert to Pandas dataframe
    df = pd.DataFrame(d, columns=columns)

    # Save to file
    df.to_pickle(pkl_path)
    df.to_csv(csv_path)

In [ ]:
import pandas as pd

df_concat = []
for s in range(0,len(snakemake.input)):
    df = pd.read_pickle(snakemake.input[s])
    df_concat.append(df)

df_concat = pd.concat(df_concat)

# Save to file
df_concat.to_pickle(snakemake.output.pkl)
df_concat.to_csv(snakemake.output.csv)

NameError: name 'snakemake' is not defined